*for more information, checkout* [Github](https://github.com/Crazytoph/2Dcorrelation "my github page").

# CD Measurements - Data Analysis

## *setting up the Notebook*

In [1]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib
import openpyxl

from scipy import integrate
import lmfit
import os

In [2]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [3]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [4]:
# change plot size
plt.rcParams["figure.figsize"] = (10,7)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [5]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\DNA Origami"
datalist = os.listdir(path)
print(datalist)

['46_forward', '46_reversed', 'Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h', 'GdmCl_4M', 'GdmCl_6M', 'GdmCl_6M_alt', 'GdmSCN_0.5M', 'GdmSCN_2M', 'Urea_2M']


In [6]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [15]:
# options to acces data
print(data_all['Control_0M'])
# multiple options
print(list(map(data_all.get, ['Control_0M', 'Gdm2SO4_4M'])))
print(tuple(data_all.values())[0:4])

[<cdata.CData object at 0x00000240D01B0EB0>, <cdata.CData object at 0x00000240D0308580>]
(<cdata.CData object at 0x00000240D01B0970>, <cdata.CData object at 0x00000240CA0D5850>, <cdata.CData object at 0x00000240D01B0EB0>, <cdata.CData object at 0x00000240CA0E84F0>)


## Print-out for PCA


In [150]:
liste = ['Control_0M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_4M', 'GdmCl_6M']

In [152]:
print_path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\PCA"
i= 1

for key in liste:
    for col in data_all[key].cd_df.loc[:, :45].columns:
        name =  print_path + '\\' + 'GdmCl_all_lowTemp' + '\\' + str(i) + '.dat'
        data_all[key].cd_df.loc[:, col].to_csv(name, sep = " ", header=False)
        i = i + 1
    print(i)

5
9
13
17
21


In [149]:
print(data_all["Control_0M"].t_list,"\n", data_all["GdmCl_0.5M"].t_list, "\n", data_all["GdmCl_2M"].t_list, "\n", data_all["GdmCl_4M"].t_list, "\n", data_all["GdmCl_6M"].t_list, "\n", melt_T)

[20, 30, 40, 45, 50, 53, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90] 
 [20, 30, 40, 45, 50, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90] 
 [20, 30, 40, 45, 50, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90] 
 [20, 30, 40, 45, 50, 53, 56, 58, 60, 62, 64, 67, 70, 75, 80, 90] 
 [20, 30, 40, 45, 50, 53, 56, 58, 60, 63, 64, 67, 70, 75, 80, 90] 
 {'46_forward': 31.606016965179656, '46_reversed': 23.78138436608932, 'Control_0M': 64.1311656504468, 'Gdm2SO4_0.5M': 65.64654525711592, 'Gdm2SO4_2M': 65.2654447695673, 'Gdm2SO4_4M': 66.31283990878158, 'Gdm2SO4_6M': 60.45861485339304, 'GdmCl_0.5M': 65.4002081960866, 'GdmCl_2M': 62.888009317411594, 'GdmCl_2M_24h': 64.97387944017076, 'GdmCl_4M': 59.751759643339646, 'GdmCl_6M': 53.22081768649669, 'GdmCl_6M_alt': -18981.919040091376, 'GdmSCN_0.5M': 64.35195325836264, 'GdmSCN_2M': 62.83358605437706, 'Urea_2M': 59.71396772380036}


## Colormaps

### Heat Maps

In [13]:
for key in data_all.keys():
    plot.heatmap(data_all[key].cd_df, subtitle=[""], title=str(key), x_min=[210], swap=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
#plot.heatmap(*tuple(data_all.values()), subtitle=tuple(data_all.keys()), title="Cd-Values", x_min=[220]*len(data_all), swap=False)

### 3D surface plots

In [14]:
df = data_all["Control_0M"].cd_df.loc[205:]

plot.function3d(df, title="")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


## Function Plots 

### CD-Spectra Plot

In [7]:
wave_min = 200
wave_max = 330

key_list = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', "Gdm2SO4_6M"]
plot.mult_func([20, 45, 64, 80], [data_all["Control_0M"].cd_df.loc[wave_min:wave_max]], swap=True,
                   x_label="Wavelength [nm]", y_label="CD values [mdeg]", subtitle=[""], marker=['', '', '', ''], linestyle=['-', ':', ':', '-'], baseline=True, label=["20°C","45°C", "64°C", "80°C"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
wave_min = 200
wave_max = 330

key_list = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'GdmCl_0.5M', 'GdmCl_2M',  'GdmCl_4M', 'GdmSCN_0.5M', 'GdmSCN_2M']
for key in key_list:
    plot.mult_func([20, 45, 64, 80], [data_all[key].cd_df.loc[wave_min:wave_max]], title=key, swap=True,
                   x_label="Wavelength [nm]", y_label="CD values [mdeg]", subtitle=[""], label=["20°C", "45°C", "64°C", "80°C"], marker=['', '', '', ''], linestyle=['-', ':', ':', '-'], baseline=True, 
                  )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
data=[]
for key in data_all.keys():
    data.append([data_all[key].cd_df.T])

plot.mult_func([20, 40, 55, 60, 65, 70, 90], *data, subtitle=tuple(data_all.keys()),
               title="CD-Spectra", marker=["","", "", "", "", "", ""], x_min=210
              )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Max/Min Plots


multiple probes

In [10]:
wave_min = 210
wave_max =  230


# First data you want to add to the plot
plot_data =  ana.max_wave(data_all["Control_0M"].cd_df, wave_min=wave_min, wave_max=wave_max)
plot_data = plot_data.drop(['Wavelength'], axis=1)   
plot_data.rename(columns = {'Value': 'Control_0M'}, inplace = True)



# other Data you want to add, if always same max/min values then loop would be possible
plot_data["GdmCl_4M"] = ana.max_wave(data_all["GdmCl_4M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
plot_data["Gdm2SO4_2M"] = ana.max_wave(data_all["Gdm2SO4_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
#plot_data["GdmSCN_2M"] 2= ana.max_wave(data_all["GdmSCN_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]


plot.mult_func(["Control_0M", "GdmCl_4M", "Gdm2SO4_2M"], [plot_data.T], subtitle=[""], title="Max. CD Value between 210nm - 230nm")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

one probe

In [15]:
wave_min = 260
wave_max =  290
title = "Max. CD-Value between " + str(wave_min) + "nm - " + str(wave_max) +"nm"

for key in data_all.keys():
    plot_data =  ana.max_wave(data_all[key].cd_df, wave_min=wave_min, wave_max=wave_max)
    plot.mult_func(["Value"], [plot_data.T], subtitle=[""], vertical_line=[melt_T[key]], title=title, label=[key, "melting Temp."])



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Correlation Analyis

<div class="alert alert-block alert-info">
<b>Info:</b> Noda, I. (2007). Two-dimensional correlation analysis useful for spectroscopy, chromatography, and other analytical measurements. Analytical Sciences, 23(2), 139–146. https://doi.org/10.2116/analsci.23.139
</div>

<div class="alert alert-block alert-info">
<b>Info:</b> Noda, I. (2015). Techniques of two-dimensional (2D) correlation spectroscopy useful in life science research. Biomedical Spectroscopy and Imaging, 4(2), 109–127. https://doi.org/10.3233/bsi-150105
</div>

### Homogenous Spectrum

*optional: pareto scaling, auto scaling and Reference and projection*

In [98]:
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M']
t_min = 20
t_max = 90

In [99]:
for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[:, 20]
    # sort out values with HT above 900
    df = data_all[key].cd_df
    for wave in data_all[key].ht_df.index:
        if data_all[key].ht_df.max(axis=1).loc[wave] >= 900:
            df = df.drop(wave)
            ref = ref.drop(wave)
    # calculate
    sync1, assync1 = ana.correlation(df, scaling='pareto', ref_spec=[ref], center=False)
    sync2, assync2 = ana.correlation(df, scaling='auto', ref_spec=[ref], center=False)
    sync3, assync3 = ana.correlation(df, scaling=None, ref_spec=[ref], center=False)
    # plot
    plot.heatmap(sync1, assync1, sync2, assync2, title=str(key), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum", "Synchronous Spectrum", "Asynchronous Spectrum"], y_label="wavelength 1 [nm]", 
                 x_label="wavenlength 2 [nm]", x_min=[215,215,215, 215], y_min=[215, 215, 215, 215])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Tools for Analysis

<div class="alert alert-block alert-info">
<b>Info:</b> Change 'key' to wanted measurement!
</div>

In [83]:
key1 = "GdmCl_2M"
key2 = "GdmCl_2M_24h"
ref1 = data_all[key1].cd_df.loc[:, 20]
ref2 = data_all[key2].cd_df.loc[:, 20]
sync, assync = ana.correlation(data_all[key1].cd_df.loc[:, :45], data_all[key2].cd_df.loc[:, :45], scaling='pareto', ref_spec=[ref1, ref2], center=False)

In [84]:
plot.heatmap(sync, assync,  title=str(key1) + str(key2), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[210, 210], y_min=[210, 210],
                y_label= str(key1) + " WL[nm]", x_label= str(key2) + " WL[nm]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Diagonal Analysis**

In [10]:
arr = sync.to_numpy()
diag =  pd.DataFrame(arr.diagonal(), index=sync.index)
wave =  list(sync.index)[:-1]
diff =  pd.DataFrame(np.diff(arr.diagonal()), index=wave)
plot.mult_func([0], [diff.T, diag.T], baseline=True, x_label="wavelength [nm]", subtitle=["Diagonal with differential"], label=["CD Values", "Differential"], x_min=210)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Line Analyis**

In [14]:
wavelength=247
# adjust wavelength to shift line analysis*

In [15]:
arr = sync.loc[wavelength, :].to_numpy()
df = pd.DataFrame(arr, index=sync.index).T
diff = pd.DataFrame(np.diff(arr), index=list(sync.index)[:-1]).T
plot.mult_func([0], [df, diff], x_label="wavelength [nm]", baseline=True, x_min=210, subtitle=["Synchronous Line Analysis"], label=[wavelength, "derivative"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Excel**

In [142]:
excel_path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Homo_out_45.xlsx"

create new document

In [143]:
with pd.ExcelWriter(excel_path, mode='w') as writer:  

                    df.to_excel(writer, sheet_name=key1 + " vs." + key2 + "async")

Homgeneous Spectra Values

In [144]:
# liste 
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h' , 'GdmCl_4M', 'GdmCl_6M', 'Urea_2M']
t_min = 20
t_max = 90



for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[:, 20]
    # sort out values with HT above 900
    df = data_all[key].cd_df
    # calculate
    sync1, assync1 = ana.correlation(df.loc[:,:45], scaling='pareto', ref_spec=[ref], center=False)
    df1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
    df2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
    with pd.ExcelWriter(excel_path, mode='a') as writer:  
                    df1.to_excel(writer, sheet_name=key  + " sync")
                    df2.to_excel(writer, sheet_name=key  + " async")

Heterogeneous Spectra Values

In [141]:
# Liste

#plot_list = ["Control_0M", 'GdmCl_0.5M', 'GdmCl_2M',  'GdmCl_4M', 'GdmCl_6M']
plot_list = ["Control_0M", 'Gdm2SO4_0.5M', 'Gdm2SO4_2M',  'Gdm2SO4_4M', 'Gdm2SO4_6M']
anzahl = len(plot_list)
t_min = 20 
t_max = 90


# calculation 
for i in range(anzahl):
    for j in range(anzahl):
        if i >= j:
            continue
        df1 = data_all[plot_list[i]].cd_df
        df2 = data_all[plot_list[j]].cd_df
        ref1 = df1.loc[:, 20]
        ref2 = df2.loc[:, 20]
        sync1, assync1 = ana.correlation(df1.loc[:, :t_max], df2.loc[:, :t_max], scaling='pareto', ref_spec=[ref1, ref2], center=False)
        df1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
        df2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
        with pd.ExcelWriter(excel_path, mode='a') as writer:  
            df1.to_excel(writer, sheet_name=plot_list[i] + " vs.  " + plot_list[j]  + " sync")
            df2.to_excel(writer, sheet_name=plot_list[i] + " vs.  " + plot_list[j]  + " async")
            
# extra calculation for 2M vs 2M_24h            
"""df1 = data_all["GdmCl_2M"].cd_df
df2 = data_all["GdmCl_2M_24h"].cd_df
ref1 = df1.loc[:, 20]
ref2 = df2.loc[:, 20]
sync1, assync1 = ana.correlation(df1.loc[:, :t_max], df2.loc[:, :t_max], scaling='pareto', ref_spec=[ref1, ref2], center=False)
df1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
df2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
with pd.ExcelWriter(excel_path, mode='a') as writer:  
    df1.to_excel(writer, sheet_name="2M vs. 2M 24h sync")
    df2.to_excel(writer, sheet_name="2M vs. 2M 24h async")"""

'df1 = data_all["GdmCl_2M"].cd_df\ndf2 = data_all["GdmCl_2M_24h"].cd_df\nref1 = df1.loc[:, 20]\nref2 = df2.loc[:, 20]\nsync1, assync1 = ana.correlation(df1.loc[:, :t_max], df2.loc[:, :t_max], scaling=\'pareto\', ref_spec=[ref1, ref2], center=False)\ndf1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)\ndf2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)\nwith pd.ExcelWriter(excel_path, mode=\'a\') as writer:  \n    df1.to_excel(writer, sheet_name="2M vs. 2M 24h sync")\n    df2.to_excel(writer, sheet_name="2M vs. 2M 24h async")'

In [74]:
arr = assync.loc[wavelength, :].to_numpy()
df = pd.DataFrame(arr, index=assync.index).T
diff = pd.DataFrame(np.diff(arr), index=list(assync.index)[:-1]).T
plot.mult_func([0], [df, diff], x_label="wavelength [nm]", baseline=True, x_min=210, subtitle=["synchronous Line Analysis"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Heterogenous Spectrum

*with pareto scaling and Control Measurement as Reference*

In [10]:
print(data_all.keys())

dict_keys(['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h', 'GdmCl_4M', 'GdmCl_6M', 'GdmSCN_0.5M', 'GdmSCN_2M', 'Urea_2M'])


In [112]:
plot_list = ["Control_0M", 'GdmCl_0.5M', 'GdmCl_2M',  'GdmCl_4M', 'GdmCl_6M']
anzahl = len(plot_list)
t_min = 20 
t_max = 90

In [113]:
for i in range(anzahl):
    for j in range(anzahl):
        if i >= j:
            continue
        title = "Heterogenoues 2D Correlation between " + plot_list[i] + " " + plot_list[j]
        df1 = data_all[plot_list[i]].cd_df
        df2 = data_all[plot_list[j]].cd_df
        ref1 = df1.loc[:, 20]
        ref2 = df2.loc[:, 20]
        sync1, assync1 = ana.correlation(df1, df2, scaling='pareto', ref_spec=[ref1, ref2], center=False)
        sync2, assync2 = ana.correlation(df1, df2, scaling='auto', ref_spec=[ref1, ref2], center=False)
        plot.heatmap(sync1, assync1, sync2, assync2, title=title, subtitle=["Synchronous Spectrum", "Asynchronous Spectrum", "Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[210, 210, 210, 210], y_min=[210, 210, 210, 210],
                y_label= plot_list[i] + " WL[nm]", x_label= plot_list[j] + " WL[nm]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Perturbation-Correlation Moving-Window 2D Correlation Spectroscopy

*different window size can be choosen*

In [16]:
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'Control_0M', 'GdmCl_0.5M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M' ]

In [23]:
for key in liste:
    sync, assync, = ana.perturbation_moving_window(data_all[key].cd_df, window_size=3)
    sync = sync.drop([200,201,202,203,204,205,206,207,208,209,210,211,212,213,214])
    assync = assync.drop([200,201,202,203,204,205,206,207,208,209,210,211,212,213,214])
    sync =  (sync-sync.min().min())/(sync.max().max()-sync.min().min())
    assync =  (assync-assync.min().min())/(assync.max().max()-assync.min().min())
    plot.heatmap(sync, x_min=[215,215], subtitle=[""], x_label="Temperature [°C]", y_label="Wavelength [nm]", title=key)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2D Correlation of Null Space Projection

*different projection_vectors/matrices can be choosen*

In [18]:
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', "Gdm2SO4_6M", "Control_0M" ,'GdmCl_0.5M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M' ]

In [21]:
for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[:, 20]
    
    # create dynamic spectrum! and calculate projection
    df = ana.projection_matrix(data_all[key].cd_df.subtract(ref, axis=0), row_min_max=[247], center=False)
    sync, assync = ana.correlation(df)
            
    plot.heatmap(sync, assync, x_min=[220, 220], y_min=[215,215], subtitle=[""], title=str(key))

C:\Users\crazy\PycharmProjects\2Dcorrelation\hotznplots.py:82: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(facecolor="white")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[:, 20]
    
    # create dynamic spectrum! and calculate projection
    df = ana.projection_matrix(data_all[key].cd_df.subtract(ref, axis=0), row_min_max=[247], center=False)
    
     # sort out values with HT above 900
    for wave in data_all[key].ht_df.index:
        if data_all[key].ht_df.max(axis=1).loc[wave] >= 900:
            df = df.drop(wave)
    # calculate
    sync1, assync1, = ana.correlation(df, ref_spec=[0])
    # plot
    plot.heatmap(sync1, assync1, title=str(key), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], y_label="wavelength [nm]", 
                 x_label="wavenlength [nm]", x_min=[220, 220], y_min=[220,220])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Sigmoid Fit and Melting Temperature

In [9]:
graph={}
melt_T={}
for key in data_all.keys():
    fit, params, error = ana.sigmoid_fit(ana.normalize(data_all[key].cd_df))
    x =  np.arange(20, 91, 0.1)
    deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
    maximum = deriv.max(axis=1) 
    T = float(deriv.idxmax(axis=1))
    maxi = pd.DataFrame({T:1}, index=["Maximum"])
    deriv = deriv.div(maximum, axis=0)
    fit = pd.concat([fit, deriv], axis=0)
    fit = pd.concat([fit, maxi], axis=0)
    graph[key] = [fit]  
    melt_T[key] = T
plot.mult_func([247, "fit", "deriv", "Maximum"], *graph.values(),  title="Sigmoid-Fit and Derivative(adjusted)",
               subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[ 0.18677066 61.81964508] [0.01970739 0.53616891]
[ 0.24733276 63.42770044] [0.02193106 0.35106698]
[ 0.3        63.95213937] [0.02830562 0.31740657]
[ 0.28173998 64.87467936] [0.02577176 0.3342244 ]
[ 0.2364997  63.66483834] [0.01583726 0.27656016]
[ 0.2939166  61.61989282] [0.01911078 0.21232322]
[ 0.26911012 63.19314219] [0.02238773 0.30450951]
[ 0.28883307 58.5239527 ] [0.02428439 0.29303091]
[1.41701334e-02 5.00000000e+01] [1.8215488e-02 2.3711382e+01]
[2.29854877e-02 7.00000000e+01] [ 0.01983874 16.65590283]
[ 0.19424656 57.00573805] [0.02250736 0.56124733]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot the melting Temperatures for the Gdm2SO4 and GdmCl

In [10]:
x = [0., 0.5, 2., 4.]
y = list(melt_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], x_label="Concentration[M]", y_label="Temperature[K]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Absorbance Values

In [94]:
absorbance = list(map(lambda x: data_all[x].absorb_df, list(data_all.keys())))
for i in absorbance:
    plot.heatmap(i, subtitle=tuple(data_all.keys()), title="Absorbance Values", x_min=[260]*len(data_all), c_min=[0]*len(data_all))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
graph={}
melt_T={}
error_a = {}
error_b = {}
coop={}
elements = ["Control_0M", "Gdm2SO4_0.5M", "Gdm2SO4_2M", "Gdm2SO4_4M", "Control_0M", "GdmCl_0.5M", "GdmCl_2M", "GdmCl_4M" ]
key = "GdmSCN_0.5M"
fit, params, std = ana.sigmoid_fit(ana.normalize(data_all[key].absorb_df), wave=260, a_range=[0.2, 0.5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph[key] = [fit]  
melt_T[key] = params[1]
error_a[key] = std[0]
error_b[key] = std[1]
coop[key] = params[0] 
plot.mult_func([260, "fit", "deriv"], graph[key],  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[ 0.2        63.29561482] [0.04565207 1.08081709]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
graph={}
melt_T={}
error_a = {}
error_b = {}
coop={}
elements = ["Control_0M", "Gdm2SO4_0.5M", "Gdm2SO4_2M", "Gdm2SO4_4M", "Control_0M", "GdmCl_0.5M", "GdmCl_2M", "GdmCl_4M" ]
for key in data_all.keys():
    fit, params, std = ana.sigmoid_fit(ana.normalize(data_all[key].absorb_df), wave=260, a_range=[0.2, 0.5])
    x =  np.arange(20, 91, 0.1)
    deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
    maximum = deriv.max(axis=1) 
    T = float(deriv.idxmax(axis=1))
    maxi = pd.DataFrame({T:1}, index=["Maximum"])
    deriv = deriv.div(maximum, axis=0)
    fit = pd.concat([fit, deriv], axis=0)
    fit = pd.concat([fit, maxi], axis=0)
    graph[key] = [fit]  
    melt_T[key] = params[1]
    error_a[key] = std[0]
    error_b[key] = std[1]
    coop[key] = params[0] 
plot.mult_func([260, "fit", "deriv"], graph["GdmCl_4M"],  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[ 0.41163539 64.15020882] [0.02314436 0.15271522]
[ 0.36215912 65.86173799] [0.02185816 0.18932737]
[ 0.44493928 65.49468303] [0.03352761 0.19683403]
[ 0.41765738 66.38498811] [0.02650862 0.17467372]
[ 0.41177395 65.56418843] [0.01837304 0.12537056]
[ 0.36338404 63.19755227] [0.02700207 0.21562398]
[ 0.38244298 64.79604632] [0.03395227 0.28041146]
[ 0.3882771  59.95701763] [0.02664735 0.18265845]
[ 0.2 50. ] [0.29051834 8.40098278]
[ 0.2        63.29561482] [0.04565207 1.08081709]
[ 0.2        62.18622927] [0.06441774 1.46286914]
[ 0.39852856 59.86735295] [0.04122087 0.2420659 ]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
x = [0., 0.5, 2., 4.]
y = list(melt_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], error=error_b, title="Melting Temperature",
               subtitle=[""], x_label="Concentration[M]", y_label="Temperature[°C]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
x = [0., 0.5, 2., 4.]
y = list(coop.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [coop["Control_0M"], coop["GdmCl_0.5M"], coop["GdmCl_2M"], coop["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], error=error_a, title="Variance",
               subtitle=[""], x_label="Concentration[M]", y_label="Temperature[K^2]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### LM Fit

In [9]:
df = data_all["Control_0M"].cd_df
plot.mult_func([280], [df], swap=False, linestyle=[""], marker=["X"], x_label="Temperature [°C]", subtitle=[""], title="Control Measurement Hyperchromatic Shift", label=["260 nm"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
liste = ["Control_0M","Gdm2SO4_0.5M", "Gdm2SO4_2M", "Gdm2SO4_4M", "Gdm2SO4_6M", "GdmCl_0.5M", "GdmCl_2M", "GdmCl_4M", "GdmCl_6M"]

In [17]:
# Parameter
sample = "Control_0M"
melt_T = {}
error_T = {}
residual = pd.DataFrame(index=["R-Wert"])
for key in liste:
    fit, params, params_error = ana.lm_fit(data_all[key].absorb_df, guess=[0.13, 0.36, 0.4])
    melt_T[key] = params['xc']
    error_T[key] = params_error
    residual[key] = [1 - (np.sum(fit["residual"]**2)/np.sum((fit[260] - fit[260].mean(axis=0))**2))]
    plot.mult_func([260, "fit"], [fit], error={"fit": fit["error"]}, swap=True, 
                   x_label="Temperature [°C]", y_label="Absorbance [%]", label=["data", "sigmoid fit", 'melting temperature'], marker=['o', ''], linestyle=[" ", "-"],
                  subtitle=["Hyperchromatic shift"], title=key, vertical_line=[params['xc']])

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 31
    # data points      = 16
    # variables        = 4
    chi-square         = 8.2538e-05
    reduced chi-square = 6.8781e-06
    Akaike info crit   = -186.797542
    Bayesian info crit = -183.707187
[[Variables]]
    a:   0.13153960 +/- 0.00186326 (1.42%) (init = 0.13)
    y0:  0.35994037 +/- 0.00101406 (0.28%) (init = 0.36)
    k:   0.46427885 +/- 0.02562654 (5.52%) (init = 0.4)
    xc:  64.0734366 +/- 0.13972373 (0.22%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -0.618
    C(a, k)   = -0.528
    C(y0, k)  =  0.374
    C(y0, xc) =  0.283
    C(a, xc)  =  0.228
    C(k, xc)  = -0.212


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 31
    # data points      = 15
    # variables        = 4
    chi-square         = 1.3729e-04
    reduced chi-square = 1.2481e-05
    Akaike info crit   = -166.021914
    Bayesian info crit = -163.189713
[[Variables]]
    a:   0.15453355 +/- 0.00274112 (1.77%) (init = 0.13)
    y0:  0.38110761 +/- 0.00145979 (0.38%) (init = 0.36)
    k:   0.39130613 +/- 0.02400150 (6.13%) (init = 0.4)
    xc:  65.4725786 +/- 0.19088378 (0.29%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -0.612
    C(a, k)   = -0.554
    C(y0, k)  =  0.405
    C(y0, xc) =  0.303
    C(a, xc)  =  0.248
    C(k, xc)  = -0.208


C:\Users\crazy\PycharmProjects\2Dcorrelation\hotznplots.py:334: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(facecolor=backgroundcolor)  # create figure


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 41
    # data points      = 16
    # variables        = 4
    chi-square         = 1.6417e-04
    reduced chi-square = 1.3681e-05
    Akaike info crit   = -175.795251
    Bayesian info crit = -172.704896
[[Variables]]
    a:   0.15628335 +/- 0.00254249 (1.63%) (init = 0.13)
    y0:  0.40464104 +/- 0.00164746 (0.41%) (init = 0.36)
    k:   0.43634504 +/- 0.02704844 (6.20%) (init = 0.4)
    xc:  59.6154686 +/- 0.15797674 (0.26%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -0.712
    C(a, k)   = -0.533
    C(y0, xc) =  0.385
    C(y0, k)  =  0.373


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 63
    # data points      = 16
    # variables        = 4
    chi-square         = 1.3882e-04
    reduced chi-square = 1.1569e-05
    Akaike info crit   = -178.478337
    Bayesian info crit = -175.387982
[[Variables]]
    a:   0.09459118 +/- 0.00303384 (3.21%) (init = 0.13)
    y0:  0.36684601 +/- 0.00222071 (0.61%) (init = 0.36)
    k:   0.22638504 +/- 0.02115257 (9.34%) (init = 0.4)
    xc:  53.0164003 +/- 0.46837083 (0.88%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -0.832
    C(a, k)   = -0.672
    C(y0, xc) =  0.551
    C(y0, k)  =  0.491
    C(a, xc)  = -0.212
    C(k, xc)  =  0.197


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
x = [0., 0.5, 2., 4., 6.]
y = list(melt_T.values())[0:5]

y_error = list(error_T.values())[0:5]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T
error= {"Gdm2SO4":y_error}

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"], melt_T["GdmCl_6M"]]
y2_error = [error_T["Control_0M"], error_T["GdmCl_0.5M"], error_T["GdmCl_2M"], error_T["GdmCl_4M"], error_T["GdmCl_6M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T
error["GdmCl"] = y2_error 

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], x_label="Concentration [M]", y_label="Temperature [°C]", error=error_T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Linear LM Fit for Melting Temperature


In [24]:
# Parameter
melt_T = {}
error_T = {}
residual = pd.DataFrame(index=["R-Wert"])
for key in data_all.keys():
    # normalize absorbance values and get next 3 values to 0.5
    df = ana.normalize(data_all[key].absorb_df).loc[260, :]
    df_sort = pd.DataFrame(df.iloc[(df-0.5).abs().argsort()[:3]], columns=[260])
    # make linear fit
    fit, params, params_error = ana.lm_fit(df_sort.T, f_type='linear')
    
    # calulate melting temperature
    melt_T[key] = (0.5 - params['y0'])/params['m']
    error_T[key] = np.sqrt((params_error['y0']/params['m'])**2 + ((0.5 - params['y0'])*params_error['m']/(params['m']**2))**2)
    residual[key] = [1 - (np.sum(fit["residual"]**2)/np.sum((fit[260] - fit[260].mean(axis=0))**2))]
    
    # make nice plot
    plot_fit = pd.DataFrame(df.values, index=df.index, columns=["260 nm"])
    plot_fit["linear fit"] = ana.linear(df.index, *params.values())
    plot.mult_func(["260 nm", "linear fit"], [plot_fit.T], title=key, subtitle=[""], y_min=[-0.05], y_max=[1.05], y_scaling=True, 
                   vertical_line=[melt_T[key]], y_label="Fraction sDnA [%]", x_label="Temperature [°C]", marker=["X", ""], label=["Hyperchromatic Shift at 260 nm", "Linear fit around 0.5", "Melting Temperature"])

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 3
    # variables        = 2
    chi-square         = 4.5580e-05
    reduced chi-square = 4.5580e-05
    Akaike info crit   = -29.2839646
    Bayesian info crit = -31.0867401
[[Variables]]
    m:   0.02751360 +/- 0.00238694 (8.68%) (init = 1)
    y0: -0.36959546 +/- 0.07648154 (20.69%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.999


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 3
    # variables        = 2
    chi-square         = 9.5661e-04
    reduced chi-square = 9.5661e-04
    Akaike info crit   = -20.1521789
    Bayesian info crit = -21.9549543
[[Variables]]
    m:  -0.00348944 +/- 0.00140010 (40.12%) (init = 1)
    y0:  0.58298370 +/- 0.05347315 (9.17%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.943


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 3
    # variables        = 2
    chi-square         = 7.4934e-04
    reduced chi-square = 7.4934e-04
    Akaike info crit   = -20.8847889
    Bayesian info crit = -22.6875644
[[Variables]]
    m:   0.10024185 +/- 0.00769145 (7.67%) (init = 1)
    y0: -5.92862655 +/- 0.49506910 (8.35%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.999


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 1.6236e-07
    reduced chi-square = 1.6236e-07
    Akaike info crit   = -46.1961442
    Bayesian info crit = -47.9989196
[[Variables]]
    m:   0.01493276 +/- 1.4246e-04 (0.95%) (init = 1)
    y0: -0.43827916 +/- 0.00883568 (2.02%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 4.0739e-06
    reduced chi-square = 4.0739e-06
    Akaike info crit   = -36.5285535
    Bayesian info crit = -38.3313290
[[Variables]]
    m:   0.09767408 +/- 7.1361e-04 (0.73%) (init = 1)
    y0: -5.33250715 +/- 0.04283247 (0.80%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
print(melt_T)

{'46_forward': 31.606016965179656, '46_reversed': 23.78138436608932, 'Control_0M': 64.1311656504468, 'Gdm2SO4_0.5M': 65.64654525711592, 'Gdm2SO4_2M': 65.2654447695673, 'Gdm2SO4_4M': 66.31283990878158, 'Gdm2SO4_6M': 60.45861485339304, 'GdmCl_0.5M': 65.4002081960866, 'GdmCl_2M': 62.888009317411594, 'GdmCl_2M_24h': 64.97387944017076, 'GdmCl_4M': 59.751759643339646, 'GdmCl_6M': 53.22081768649669, 'GdmCl_6M_alt': -18981.919040091376, 'GdmSCN_0.5M': 64.35195325836264, 'GdmSCN_2M': 62.83358605437706, 'Urea_2M': 59.71396772380036}


In [21]:
x = [0., 0.5, 2., 4., 6.]
y =  [melt_T["Control_0M"], melt_T["Gdm2SO4_0.5M"], melt_T["Gdm2SO4_2M"], melt_T["Gdm2SO4_4M"], melt_T["Gdm2SO4_6M"]]

y_error = [error_T["Control_0M"], melt_T["Gdm2SO4_0.5M"], melt_T["Gdm2SO4_2M"], melt_T["Gdm2SO4_4M"], melt_T["Gdm2SO4_6M"]]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T
error= {"Gdm2SO4":y_error}

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"], melt_T["GdmCl_6M"]]
y2_error = [error_T["Control_0M"], error_T["GdmCl_0.5M"], error_T["GdmCl_2M"], error_T["GdmCl_4M"], error_T["GdmCl_6M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T
error["GdmCl"] = y2_error 

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], x_label="Concentration [M]", y_label="Temperature [°C]", error=error_T)y_scaling=vertical_line=

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Special for GdmSCN**

In [26]:
data_all[key].t_list

[20, 30, 40, 45, 50, 53, 56, 58, 60, 63, 64, 67, 70, 75, 80, 90]

In [39]:
for key in data_all.keys():

    # fit three secitons
    df =  data_all[key].absorb_df
    high_fit, high_params, high_std =  ana.lm_fit(df.iloc[:,-3:], f_type='linear')
    low_fit, low_params, low_std =  ana.lm_fit(df.iloc[:,:3], f_type='linear')


    # normalize absorbance values and get next 3 values to 0.5
    df_norm = ana.normalize(df).loc[260, :]
    df_sort = pd.DataFrame(df_norm.iloc[(df_norm-0.5).abs().argsort()[:3]], columns=[260])
    # make linear fit
    middle_fit, middle_params, middle_std = ana.lm_fit(df.loc[:, df_sort.index], f_type='linear')


    # calculate cross of middle and median
    t_melt = ( ( (- high_params["y0"] - low_params["y0"]) / 2 ) + middle_params["y0"] ) / ( ( (high_params["m"] + low_params["m"]) / 2 ) - middle_params["m"] ) 
    # calculate error 
    teiler = ( ( (high_params["m"] + low_params["m"]) / 2 ) - middle_params["m"] ) 
    nenner =  ( ( (- high_params["y0"] - low_params["y0"]) / 2 ) + middle_params["y0"] ) 
    t_melt_error =  np.sqrt( (-0.5 * high_std["y0"] / teiler)**2 + (-0.5 * low_std["y0"] / teiler)**2 + (middle_std["y0"] / teiler)**2
                              + (nenner * 0.5 * high_std["m"] / (teiler**2) )**2 + (nenner * 0.5 * low_std["m"] / (teiler**2) )**2 + (nenner * -1 * middle_std["m"] / (teiler**2) )**2)

    # make nice plot
    plot_fit= pd.DataFrame(ana.linear(df.columns, *high_params.values()), columns=["high plateu"], index=df.columns)
    plot_fit["low plateu"] = ana.linear(df.columns, *low_params.values())
    plot_fit["middle"] = ana.linear(df.columns, *middle_params.values())
    plot_fit["Median"] = 0.5*(plot_fit["high plateu"] + plot_fit["low plateu"])

    plot.mult_func([260, "high plateu", "low plateu", "middle", "Median"], [df, plot_fit.T], 
                   vertical_line=[t_melt], marker=["X", "", "", "", ""], linestyle=["--", ":", ":", "-.", "-."],
                   y_scaling=True, y_min=[df.min(axis=1).loc[260] - 0.02], y_max=[df.max(axis=1).loc[260] + 0.02], 
                   title=key, subtitle=[""], sec_ax=[df.min(axis=1).loc[260], df.max(axis=1).loc[260]])
    print(melt_T[key], t_melt, t_melt_error)

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 1.9267e-10
    reduced chi-square = 1.9267e-10
    Akaike info crit   = -66.4060155
    Bayesian info crit = -68.2087909
[[Variables]]
    m:   5.3600e-04 +/- 4.9075e-06 (0.92%) (init = 1)
    y0:  0.28275033 +/- 2.1608e-04 (0.08%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.999
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 1.2422e-08
    reduced chi-square = 1.2422e-08
    Akaike info crit   = -53.9073479
    Bayesian info crit = -55.7101233
[[Variables]]
    m:   6.8525e-04 +/- 3.9404e-05 (5.75%) (init = 1)
    y0:  0.27756650 +/- 7.1219e-04 (0.26%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'list' object has no attribute 'keys'

# Long measurements

In [34]:
df = data_all["46_forward"].absorb_df
deriv_df = ana.derivative(df)
deriv_df.columns = deriv_df.columns + df.columns[0]

plot.mult_func([260], [df])
plot.heatmap(df, x_min=[220] )



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: Shape of passed values is (131, 16), indices imply (131, 1)

In [26]:
plot.mult_func([215], [data_all['46_forward'].cd_df, data_all['46_reversed'] ], swap=False,
                   y_label="CD values [mdeg]", subtitle=[""], marker=['o', 'x', '', ''],
               linestyle=['-', ':', ':', '-'], label=["Forward", "Reversed"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# TD Calculations


<div class="alert alert-block alert-info">
<b>Info:</b> Steps after Mergny, J.-L., & Lacroix, L. (2003). Analysis of Thermal Melting Curves. In OLIGONUCLEOTIDES (Vol. 13). www.liebertpub.com
</div>

**Hypochromatic Shift at 260nm  wit Low and High Plateau Fit**

In [24]:
raw = data_all["Control_0M"].absorb_df
low_fit, params_lowfit = ana.lm_fit(raw[[20,30,40]], f_type='linear', guess=[0.0001, 0.35])
high_fit, params_highfit = ana.lm_fit(raw[[70, 80,90]], f_type='linear', guess=[0.0001, 0.48])
sig_fit, params_sigfit = ana.lm_fit(raw, f_type='s1', guess=[0.2, 0.35, 0.5, 63])

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 6
    # variables        = 2
    chi-square         = 1.6258e-04
    reduced chi-square = 4.0646e-05
    Akaike info crit   = -59.0964195
    Bayesian info crit = -59.5129005
[[Variables]]
    m:   0.00432721 +/- 2.2635e-04 (5.23%) (init = 0.004)
    y0:  0.28550310 +/- 0.00934823 (3.27%) (init = 0.2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.960
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 1.8115e-05
    reduced chi-square = 1.8115e-05
    Akaike info crit   = -32.0521993
    Bayesian info crit = -33.8549747
[[Variables]]
    m:   0.01084090 +/- 3.9404e-04 (3.63%) (init = 0.01)
    y0: -0.02179150 +/- 0.03227374 (148.10%) (init = -0.02)
[[Correlations]] (unreported correlations are < 0.

**Calculating Fraction Folded with High Plateu adjusted manually**

In [25]:
data_treated = pd.DataFrame(raw.loc[260])
data_treated["low_fit"] = ana.linear(np.array(raw.columns), *params_lowfit.values())
data_treated["high_fit"] = ana.linear(np.array(raw.columns), 0.0004495999948082116, 0.45451300041966917)
data_treated["intermediate"] = 0.5*(data_treated["low_fit"] + data_treated["high_fit"])
data_treated["fit"] = sig_fit["fit"]
data_treated["baseline_fit"] = (data_treated["high_fit"] - data_treated[260])/(data_treated["high_fit"] - data_treated["low_fit"])
data_treated["normed_data"] = 1- ana.normalize(raw).loc[260].to_numpy()
data_treated["log_fit"] = 1 - ana.normalize(data_treated)["fit"]
plot.mult_func([260, "low_fit", "high_fit", "intermediate", "fit"], [data_treated.T], subtitle=[""], title="Hypochromatic shift with Plaetau Fits", y_label="Absorbance at 260nm")
plot.mult_func(["baseline_fit", "normed_data", "log_fit"], [data_treated.T], subtitle=[""],  title="Fraction folded calculated by Plateu fits and Normalization", y_label="Fraction folded")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**calulate K and ln K for different methods**

In [10]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
base_method = pd.DataFrame(data_treated["baseline_fit"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
base_method['theta'] = base_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
base_method["K"] = (base_method["theta"])/(c_total*((1 - base_method["theta"])**2))
base_method['ln K'] = np.log(base_method['K'])

# select values in confidence values 0.1 < theta < 0.9
base_method  = base_method.loc[60:70]
# change celsius to kelvin
base_method = base_method.set_index(1/(base_method.index + 273.15))

# make linear fit and plot
enthalpy_base, params_base = ana.lm_fit(base_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_base["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_base.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 5
    # variables        = 2
    chi-square         = 0.66852599
    reduced chi-square = 0.22284200
    Akaike info crit   = -6.06058962
    Bayesian info crit = -6.84171380
[[Variables]]
    m:   82573.1995 +/- 6790.09991 (8.22%) (init = 80000)
    y0: -226.953410 +/- 20.1072530 (8.86%) (init = -2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
normed_method = pd.DataFrame(data_treated["normed_data"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
normed_method['theta'] = normed_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
normed_method["K"] = (normed_method["theta"])/(c_total*((1 - normed_method["theta"])**2))
normed_method['ln K'] = np.log(normed_method['K'])

# select values in confidence values 0.1 < theta < 0.9
normed_method  = normed_method.loc[60:70]
# change celsius to kelvin
normed_method = normed_method.set_index(1/(normed_method.index + 273.15))

# make linear fit and plot
enthalpy_normed, params_norm = ana.lm_fit(base_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_normed["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_normed.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 5
    # variables        = 2
    chi-square         = 0.66852599
    reduced chi-square = 0.22284200
    Akaike info crit   = -6.06058962
    Bayesian info crit = -6.84171380
[[Variables]]
    m:   82573.1995 +/- 6790.09991 (8.22%) (init = 80000)
    y0: -226.953410 +/- 20.1072530 (8.86%) (init = -2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
fit_method = pd.DataFrame(data_treated["log_fit"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
fit_method['theta'] = fit_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
fit_method["K"] = (fit_method["theta"])/(c_total*((1 - fit_method["theta"])**2))
fit_method['ln K'] = np.log(fit_method['K'])

# select values in confidence values 0.1 < theta < 0.9
fit_method  = fit_method.loc[60:70]
# change celsius to kelvin
fit_method = fit_method.set_index(1/(fit_method.index + 273.15))

# make linear fit and plot
enthalpy_fit, params_fit = ana.lm_fit(fit_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_fit["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_fit.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 5
    # variables        = 2
    chi-square         = 1.62690318
    reduced chi-square = 0.54230106
    Akaike info crit   = -1.61379796
    Bayesian info crit = -2.39492213
[[Variables]]
    m:   143015.744 +/- 10592.4844 (7.41%) (init = 80000)
    y0: -405.548597 +/- 31.3671034 (7.73%) (init = -2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Calculate Entropy and Enthalpy**

In [13]:
enthalpy = pd.DataFrame(["H [kJ/mol]", "H [kcal/mol]", "S [kJ/mol*K]", "S [kcal/mol*K]"], columns=["units"])
enthalpy = enthalpy.set_index("units")

# for base method
m, y0 = params_base.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["base"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

# for normed method
m, y0 = params_norm.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["norm"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

# for fit method
m, y0 = params_fit.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["fit"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

print(enthalpy)

                      base        norm          fit
units                                              
H [kJ/mol]     -686.554867 -686.554867 -1189.104403
H [kcal/mol]   -164.090527 -164.090527  -284.202730
S [kJ/mol*K]     -1.887004   -1.887004    -3.371934
S [kcal/mol*K]   -0.451005   -0.451005    -0.805911


## Derivative


In [14]:
deriv = ana.derivative(data_all["Control_0M"].absorb_df)
data = data_all["Control_0M"].absorb_df

In [15]:
plot.mult_func([260], [data], [deriv], title= "Hypochromatic shift", subtitle=["Absolute value", "interpolated derivative"], label=[""], y_label="Absorbance")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Test

**Beers Law**

In [ ]:
'Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h', 'GdmCl_4M', 'GdmCl_6M', 

In [11]:
plot.mult_func([280], [data_all['Control_0M'].cd_df, data_all['GdmCl_0.5M'].cd_df, data_all['GdmCl_2M'].cd_df, data_all['GdmCl_4M'].cd_df, data_all['GdmCl_6M'].cd_df],
               marker=["","","","","",""], linestyle=[":",":",":",":",":",":"], label=["0M", "0.5M", "2M", "4M", "6M"])

C:\Users\crazy\PycharmProjects\2Dcorrelation\hotznplots.py:336: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(facecolor=backgroundcolor)  # create figure


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
for key in data_all.keys():
    absorbance = data_all[key].absorb_df.loc[260, 20]
    df = data_all[key].cd_df
    
    conc_ds =  absorbance/(0.02*0.1) #[µg/mL]
    molar = conc_ds * 10**(-6) / (4472760.4 * 10**(-3)) #[mol/l]
    df_norm = 100 * df / (molar * 0.1)
    print(key, ":  \t", molar)

46_forward :  	 3.255551090999643e-08
46_reversed :  	 3.4156088486206414e-08
Control_0M :  	 3.9915283635582175e-08
Gdm2SO4_0.5M :  	 4.2751787017252254e-08
Gdm2SO4_2M :  	 4.442346162785737e-08
Gdm2SO4_4M :  	 4.497535794673911e-08
Gdm2SO4_6M :  	 4.012164389579196e-08
GdmCl_0.5M :  	 4.2438669417659834e-08
GdmCl_2M :  	 3.8402794837836595e-08
GdmCl_2M_24h :  	 4.286480447287093e-08
GdmCl_4M :  	 4.529876002300503e-08
GdmCl_6M :  	 4.0553927279449166e-08
GdmCl_6M_alt :  	 3.584687880888947e-08
GdmSCN_0.5M :  	 4.230485943311428e-08
GdmSCN_2M :  	 4.815627950918185e-08
Urea_2M :  	 4.0489090361290087e-08


In [83]:
for key in data_all.keys():
    plot.mult_func([280], [data_all[key].cd_df], subtitle=[key], y_scaling=True, y_max=[3.85], y_min=[1.75])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:


        # put second y_axis for "GdmCl_0.5M" normed
        # function to bring back values
        def expand_back(y):
            return y * (0.523893 - 0.379531) + 0.379531
        # inverse function
        def norm(y):
            return (y - 0.379531) / (0.523893 - 0.379531)

        # put axis and label
        secax = ax.secondary_yaxis('right', functions=(expand_back, norm), fontsize=16)
        secax.set_ylabel('Absorption [%]', fontsize=16)

In [8]:
df = data_all["Control_0M"].cd_df
mean = df.mean(axis=1)
var =  df.var(axis=1)
dyn = df.subtract(mean, axis=0)

col = dyn.columns 
idx = dyn.index
m = len(col)
n =len(idx)

In [53]:
wave1 =  247
wave2 = 260
i = 20

print(np.sum((dyn.loc[wave1, :]/mean[wave1] - dyn.loc[wave2, :]/mean[wave2]) * col) * np.sqrt(var.loc[wave1]*var.loc[wave2])/(m*(m-1)) )


-0.32785233454543217


In [9]:

assync = pd.DataFrame(np.zeros((n,n)), index=idx, columns=idx)

for j in idx:
    for k in idx:
        assync.loc[j,k] = np.sum((dyn.loc[j, :]/mean[j] - dyn.loc[k, :]/mean[k]) * col) * np.sqrt(var.loc[j]*var.loc[k])/(m*(m-1))
print(assync)

wavelength        200        201       202       203       204       205  \
wavelength                                                                 
200          0.000000  -1.631021 -1.559954 -1.176642 -0.857789 -1.347762   
201          1.631021   0.000000 -0.216200 -0.197855 -0.205062 -0.614957   
202          1.559954   0.216200  0.000000 -0.033265 -0.082423 -0.409509   
203          1.176642   0.197855  0.033265  0.000000 -0.043878 -0.280145   
204          0.857789   0.205062  0.082423  0.043878  0.000000 -0.153970   
205          1.347762   0.614957  0.409509  0.280145  0.153970  0.000000   
206         -1.171317  -0.996270 -0.797596 -0.576634 -0.376694 -0.381617   
207         -0.233238  -0.435472 -0.385581 -0.285863 -0.199700 -0.271904   
208          0.158703  -0.188669 -0.201486 -0.155361 -0.119178 -0.215740   
209          0.319298  -0.127932 -0.164682 -0.131025 -0.107426 -0.226102   
210          0.218335  -0.221386 -0.240681 -0.186197 -0.143882 -0.265258   
211         

In [11]:
plot.heatmap(assync, x_min=[220], y_min=[220])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Parafak Ethanol

## *setting up the Notebook*

In [9]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib

from scipy import integrate
import lmfit
import os

In [10]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [11]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [12]:
# change plot size
plt.rcParams["figure.figsize"] = (11,9)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [13]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\Ethanol"
datalist = os.listdir(path)
print(datalist)

['Control_0%', 'Ethanol_50%', 'Ethanol_50%_nachTest', 'Ethanol_50%_real', 'Ethanol_50%_vorTest']


In [14]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [15]:
df =  data_all["Control_0%"].cd_df
df = df.rename(columns={-1:"0%"})
df["50%"] =  (data_all["Ethanol_50%"].cd_df.loc[:,-1])
df["nach"] =  (data_all["Ethanol_50%_nachTest"].cd_df.loc[:,-1])
df["vor"] =  (data_all["Ethanol_50%_vorTest"].cd_df.loc[:,-1])
df["50% real"] =  (data_all["Ethanol_50%_real"].cd_df.loc[:,-1])
print(df)

                  0%       50%      nach       vor  50% real
wavelength                                                  
220         2.018260  2.274510  3.126240  2.307520  2.848860
221         2.080980  2.351290  3.314980  2.367540  2.935160
222         2.050420  2.385820  3.392230  2.399380  2.992490
223         1.912300  2.286210  3.399130  2.399530  2.952990
224         1.639410  2.188990  3.234820  2.326470  2.905010
225         1.334610  2.006050  2.974400  2.194240  2.709800
226         0.987198  1.847160  2.640560  1.936590  2.520000
227         0.635405  1.639230  2.248080  1.665910  2.251200
228         0.240217  1.421670  1.923790  1.448930  1.967390
229        -0.119267  1.181210  1.646120  1.259320  1.597030
230        -0.512917  0.921575  1.261840  1.015550  1.176200
231        -0.886912  0.619775  0.926121  0.713306  0.785832
232        -1.336630  0.354009  0.370603  0.331813  0.390593
233        -1.776360  0.074562  0.018606  0.083282  0.037718
234        -2.209810 -0.

In [16]:
plot.mult_func(['0%', 'vor', 'nach', '50%', "50% real" ], [df], swap=True, marker=["", "", "", "", ""], x_label="Wavelengt [nm]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …